# Final Project Sketch :(


In [6]:
import pandas as pd
import numpy as np
import os
from math import sin, cos, sqrt, atan2, radians
import time

pd.set_option('display.max_columns', 500)
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="E:/GitHub/NYU-Taxi-project/My First Project-b3735f3b3a9d.json"
from google.cloud import bigquery
client = bigquery.Client()

## Query Data from BigQuery

In [4]:
start = time.time()
query = (
"""
SELECT date_of_year, month_of_year, day_of_year, pickup_hour,
    dropoff_datetime,pickup_datetime,
    pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude, travel_time,
    temp, visib,wdsp,gust,max,min,prcp,sndp,fog,rain_drizzle,snow_ice_pellets,hail,thunder
FROM
(
      SELECT *,ROW_NUMBER() OVER(partition by date_of_year, pickup_hour) as row_number
      FROM
      (
            SELECT pickup_datetime, dropoff_datetime, pickup_latitude, pickup_longitude, dropoff_latitude, dropoff_longitude,
            EXTRACT (HOUR FROM pickup_datetime) as pickup_hour,
            EXTRACT (DAY FROM pickup_datetime) as day_of_year,
            EXTRACT (MONTH FROM pickup_datetime) as month_of_year,
            EXTRACT (DATE FROM pickup_datetime) as date_of_year,
            TIMESTAMP_DIFF(TIMESTAMP(dropoff_datetime), TIMESTAMP(pickup_datetime), MINUTE) as travel_time
            FROM `bigquery-public-data.new_york_taxi_trips.tlc_yellow_trips_2016`
            WHERE pickup_latitude IS NOT NULL AND pickup_longitude IS NOT NULL and
                dropoff_longitude IS NOT NULL and dropoff_latitude IS NOT NULL
      ) x
) x
JOIN
(
 select cast(mo as int64) as month,cast(da as int64) as day,temp,visib,mxpsd,wdsp,gust,max,min,prcp,sndp,fog,rain_drizzle,snow_ice_pellets,hail,thunder
 from `bigquery-public-data.noaa_gsod.gsod2016` where stn='725053'
) y on x.month_of_year = y.month and x.day_of_year = y.day
where x.row_number <= 200
"""
)
data=pd.io.gbq.read_gbq(query,dialect='standard')
end = time.time()

In [5]:
data.head()

,date_of_year,month_of_year,day_of_year,pickup_hour,dropoff_datetime,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,travel_time,temp,visib,wdsp,gust,max,min,prcp,sndp,fog,rain_drizzle,snow_ice_pellets,hail,thunder
0,2016-01-01,1,1,11,2016-01-01 11:10:51,2016-01-01 11:03:50,-73.968033,40.759506,-73.986877,40.735752,7,40.4,10.0,6.5,22.9,48.0,37.0,0.03,999.9,0,0,0,0,0
1,2016-01-01,1,1,11,2016-01-01 12:10:58,2016-01-01 11:57:46,-73.997055,40.722179,-73.977394,40.758743,13,40.4,10.0,6.5,22.9,48.0,37.0,0.03,999.9,0,0,0,0,0
2,2016-01-01,1,1,11,2016-01-01 11:35:34,2016-01-01 11:26:49,-73.983940,40.775303,-73.975273,40.759426,8,40.4,10.0,6.5,22.9,48.0,37.0,0.03,999.9,0,0,0,0,0
3,2016-01-01,1,1,11,2016-01-01 11:33:35,2016-01-01 11:30:19,0.000000,0.000000,0.000000,0.000000,3,40.4,10.0,6.5,22.9,48.0,37.0,0.03,999.9,0,0,0,0,0
4,2016-01-01,1,1,11,2016-01-01 11:33:30,2016-01-01 11:31:48,-73.967598,40.755981,-73.976486,40.743851,1,40.4,10.0,6.5,22.9,48.0,37.0,0.03,999.9,0,0,0,0,0


## Calculate Distance 

In [78]:
R = 6373.0

dlon = np.radians(data['dropoff_longitude']) - np.radians(data['pickup_longitude'])
dlat = np.radians(data['dropoff_latitude']) - np.radians(data['pickup_latitude'])

#a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
#c = 2 * atan2(sqrt(a), sqrt(1 - a))

a = np.sin(dlat / 2)**2 + np.cos(data['pickup_latitude']) * np.cos(data['pickup_longitude']) * np.sin(dlon / 2)**2
c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))

data['distance_in_km'] = R*c
data.head()

E:\Program Files\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: RuntimeWarning: invalid value encountered in sqrt
  # Remove the CWD from sys.path while we load stuff.


,date_of_year,month_of_year,day_of_year,pickup_hour,dropoff_datetime,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,travel_time,temp,visib,wdsp,gust,max,min,prcp,sndp,fog,rain_drizzle,snow_ice_pellets,hail,thunder,distance_in_km
0,2016-01-10,1,10,21,2016-01-10 21:45:10,2016-01-10 21:18:51,-73.863174,40.768604,-73.940155,40.688240,26,50.4,5.3,8.2,26.0,59.0,39.9,0.77,999.9,0,1,0,0,0,8.791497
1,2016-01-10,1,10,21,2016-01-10 21:59:17,2016-01-10 21:57:13,-73.989243,40.762913,-73.996712,40.752739,2,50.4,5.3,8.2,26.0,59.0,39.9,0.77,999.9,0,1,0,0,0,1.081536
2,2016-01-10,1,10,21,2016-01-10 21:35:36,2016-01-10 21:19:42,-73.978012,40.773800,-74.008682,40.732346,15,50.4,5.3,8.2,26.0,59.0,39.9,0.77,999.9,0,1,0,0,0,4.418026
3,2016-01-10,1,10,21,2016-01-10 21:06:59,2016-01-10 21:01:05,-73.978355,40.752007,-73.982719,40.762005,5,50.4,5.3,8.2,26.0,59.0,39.9,0.77,999.9,0,1,0,0,0,1.096132
4,2016-01-10,1,10,21,2016-01-10 22:05:19,2016-01-10 21:55:27,-74.001099,40.741619,-73.987320,40.765430,9,50.4,5.3,8.2,26.0,59.0,39.9,0.77,999.9,0,1,0,0,0,2.571153


## Summary of Data

## Optimize the memory usage

In [9]:
# Create a function to be calculating memory usage
def get_usage(obj):
    if isinstance(obj, pd.DataFrame):
        usage = obj.memory_usage(deep=True).sum()
    else:
        usage = obj.memory_usage(deep=True)
    return "{:03.2f} MB".format(usage / (1024**2))

# Convert int64 to int8
obj = data.select_dtypes(include = 'int64')
obj_int = obj.apply(pd.to_numeric, downcast='unsigned')

# Convert float64 to float32
obj = data.select_dtypes(include = 'float64')
obj_f = obj.apply(pd.to_numeric, downcast='float')

# Calculated the memory we reduced by down grade int and float
data_opt = data.copy()
data_opt[obj_int.columns] = obj_int
data_opt[obj_f.columns] = obj_f
print("Original memory usage: ", get_usage(data))
print("After converting int and float memory usage: ", get_usage(data_opt))

Original memory usage:  2133.59 MB
After converting int and float memory usage:  1880.43 MB


## Process Missing Data & Outlier

In [63]:
data_small = data.loc[data['distance_in_km'] > 0.1,]

In [69]:
data.loc[(data['travel_time']>0) & (data['distance_in_km']<50)  & (data['travel_time']<1000)]

KeyError: 'travel_time'

## EDA

In [67]:
%matplotlib inline
start = time.time()
sns.set(style="white", palette="muted", color_codes=True)
f, axes = plt.subplots(1, 1, figsize=(11, 7), sharex=True)
sns.despine(left=True)
sns.distplot(np.log(train_df['trip_duration'].values+1), axlabel = 'Log(trip_duration)', label = 'log(trip_duration)', bins = 50, color="r")
plt.setp(axes, yticks=[])
plt.tight_layout()
end = time.time()
print("Time taken by above cell is {}.".format((end-start)))
plt.show()

NameError: name 'time' is not defined

In [54]:
sum(np.isnan(data['distance_in_km'] ))

159312

In [80]:
system.time()

NameError: name 'system' is not defined

In [79]:
import plotly.plotly as py
import plotly.graph_objs as go
import plotly
plotly.tools.set_credentials_file(username='zxf71699', api_key='BkltlyEFcOkOAgCSdkjt')